In [1]:
%cd /teamspace/studios/this_studio/Deeplab/
from inference.infer import *
from post_processing.control_random_field import *

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/teamspace/studios/this_studio/Deeplab


In [2]:
from dotenv import load_dotenv

env_path = '/teamspace/studios/this_studio/Deeplab/credentials.env'
load_dotenv(dotenv_path=env_path)
import boto3

In [3]:
model = network.modeling.__dict__['deeplabv3plus_mobilenet'](num_classes= 1, output_stride= 16)

In [4]:
print(model)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (low_level_features): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inhouse_default = '/teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth'
#model = load_deeplab_model(inhouse_default, device).eval()

In [5]:
    network.convert_to_separable_conv(model.classifier)
    utils.set_bn_momentum(model.backbone, momentum=0.01)
    checkpoint = torch.load(inhouse_default, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint["model_state"])

<All keys matched successfully>

In [6]:
from torch import nn

class DeepLabFeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(DeepLabFeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
    
    def forward(self, x):
        x = self.features(x)
        return x

feature_extractor = DeepLabFeatureExtractor(model)
#print(feature_extractor) # Set the model to evaluation mode


In [7]:
    feature_extractor = nn.DataParallel(feature_extractor)
    feature_extractor.to(device)

DataParallel(
  (module): DeepLabFeatureExtractor(
    (features): Sequential(
      (0): IntermediateLayerGetter(
        (low_level_features): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): InvertedResidual(
            (conv): Sequential(
              (0): ConvBNReLU(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU6(inplace=True)
              )
              (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (2): BatchNorm2d(16, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
            )
          )
          (2): InvertedResidual(
  

In [18]:
    img = Image.open('/teamspace/studios/this_studio/Deeplab/inhouse_test/samples/test_1.jpg').convert('RGB')
    img_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((512, 512)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_tensor = img_transform(img).unsqueeze(0).to(device, dtype=torch.float32)

    with torch.no_grad():
        output_1 = feature_extractor(img_tensor)

In [17]:
img = Image.open('/teamspace/studios/this_studio/Deeplab/inhouse_test/samples/test_1.jpg').convert('RGB')
print(img.size)

(435, 618)


In [19]:
    img = Image.open('/teamspace/studios/this_studio/Deeplab/crf_sam_annotations/samples/eastafrica_acactort_2020.05.08.14.52.18_820f49c8-795b-453e-8eb1-7d473669ab84_img_20200507_152251_66198747.jpg').convert('RGB')
    img_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((512, 512)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_tensor = img_transform(img).unsqueeze(0).to(device, dtype=torch.float32)

    with torch.no_grad():
        output_2 = feature_extractor(img_tensor)

In [11]:
low_level_features = output_1['low_level'].cpu().numpy()

In [13]:
print(type(low_level_features))

<class 'numpy.ndarray'>


In [20]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
features_1_flat = output_1['low_level'].cpu().numpy().reshape(output_1['low_level'].cpu().numpy().shape[0], -1)
features_2_flat = output_2['low_level'].cpu().numpy().reshape(output_2['low_level'].cpu().numpy().shape[0], -1)

In [21]:
distance_matrix = euclidean_distances(features_1_flat, features_2_flat)

In [22]:
print(distance_matrix)

[[711.4364]]


In [23]:
def get_distance(img_1_path,img_2_path):
    img_1 = Image.open(img_1_path)
    img_2 = Image.open(img_2_path)
    img_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((512, 512)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img_1 = img_transform(img_1).unsqueeze(0).to(device, dtype=torch.float32)
    img_2 = img_transform(img_2).unsqueeze(0).to(device, dtype=torch.float32)
    with torch.no_grad():
        output_1 = feature_extractor(img_1)
        output_2 = feature_extractor(img_2)
    features_1_flat = output_1['low_level'].cpu().numpy().reshape(output_1['low_level'].cpu().numpy().shape[0], -1)
    features_2_flat = output_2['low_level'].cpu().numpy().reshape(output_2['low_level'].cpu().numpy().shape[0], -1)
    return euclidean_distances(features_1_flat, features_2_flat)

In [34]:
print(get_distance('/teamspace/studios/this_studio/Deeplab/inhouse_india/samples/test_1.jpg','/teamspace/studios/this_studio/Deeplab/inhouse_india/samples/test_9.jpg'))


[[709.17694]]


In [ ]:
print(output)

In [16]:
    network.convert_to_separable_conv(model.classifier)
    
    # Set batch norm momentum
    utils.set_bn_momentum(model.backbone, momentum=0.01)
    
    # Load the checkpoint
    checkpoint = torch.load(inhouse_default, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint["model_state"])
    print("Resume model from %s" % inhouse_default)
    
    # Create the feature extractor

    
    # Wrap in DataParallel and move to device
    feature_extractor = nn.DataParallel(feature_extractor)
    feature_extractor.to(device)

RuntimeError: Error(s) in loading state_dict for DeepLabV3:
	Missing key(s) in state_dict: "classifier.aspp.convs.1.0.body.0.body.0.weight", "classifier.aspp.convs.1.0.body.0.body.1.weight", "classifier.aspp.convs.2.0.body.0.body.0.weight", "classifier.aspp.convs.2.0.body.0.body.1.weight", "classifier.aspp.convs.3.0.body.0.body.0.weight", "classifier.aspp.convs.3.0.body.0.body.1.weight", "classifier.classifier.0.body.0.body.0.weight", "classifier.classifier.0.body.0.body.1.weight". 
	Unexpected key(s) in state_dict: "classifier.aspp.convs.1.0.body.0.weight", "classifier.aspp.convs.2.0.body.0.weight", "classifier.aspp.convs.3.0.body.0.weight", "classifier.classifier.0.body.0.weight". 